In [2]:
from opt_einsum import contract
import numpy as np
import torch
from dataclasses import asdict
from tqdm.auto import tqdm
from utils import show_tensor_ijkl
import matplotlib.pyplot as plt
from copy import deepcopy
import itertools as itt
import os
device=torch.device('cuda:0')
torch.set_default_tensor_type(torch.cuda.DoubleTensor)
torch.cuda.set_device(device)

import importlib
import MERA
importlib.reload(MERA)
from MERA import *

In [3]:
def get_Ising1D_Hamiltonians(J,g):
    sZ=torch.tensor([[1,0],[0,-1]])
    sX=torch.tensor([[0,1],[1,0]])
    eye=torch.eye(2)
    ZZ=contract('iI,jJ->ijIJ',sZ,sZ)
    XI=contract('iI,jJ->ijIJ',sX,eye)
    IX=contract('iI,jJ->ijIJ',eye,sX)
    h=-J*(ZZ+g*(XI+IX)/2)
    return h,h.clone()

def get_Ising1D_Hamiltonians_2blocked(J,g):
    sZ=torch.tensor([[1,0],[0,-1]])
    sX=torch.tensor([[0,1],[1,0]])
    eye=torch.eye(2)
    ZZII=contract('iI,jJ,kK,lL->ijklIJKL',sZ,sZ,eye,eye)
    IZZI=contract('iI,jJ,kK,lL->ijklIJKL',eye,sZ,sZ,eye)
    IIZZ=contract('iI,jJ,kK,lL->ijklIJKL',eye,eye,sZ,sZ)
    XIII=contract('iI,jJ,kK,lL->ijklIJKL',sX,eye,eye,eye)
    IXII=contract('iI,jJ,kK,lL->ijklIJKL',eye,sX,eye,eye)
    IIXI=contract('iI,jJ,kK,lL->ijklIJKL',eye,eye,sX,eye)
    IIIX=contract('iI,jJ,kK,lL->ijklIJKL',eye,eye,eye,sX)
    h=((-J*((ZZII+IIZZ)/2+IZZI)-J*g*(XIII+IXII+IIXI+IIIX)/2)/2).reshape(2,2,2,2,2,2,2,2)
    # need to take care of the reflection symmetry!
    hAB=contract('abcdABCD->abdcABDC',h).reshape(4,4,4,4)
    hBA=contract('abcdABCD->bacdBACD',h).reshape(4,4,4,4)
    return hAB, hBA


energy_ref=-4/np.pi
scdim_ref=[0,.125,1]+[1.125]*2+[2]*4+[2.125]*3+[3]*5+[3.125]*6+[4]*9+[4.125]*9+[5]*13+[5.125]*14

def troubleshoot(layers:'list[MERALayer]',options:MERAOptions,verbose=0):
    global energy_ref, scdim_ref
    energy=get_energy(layers[0],options).detach().cpu().item()
    energyErr=abs(energy-energy_ref)
    print('Energy', '%.7f'%energy, 'error', '%.2e'%energyErr)
    if verbose>0:
        ops,scdims=get_conformal_ops(layers[-1],k=10)
        print('scdims', scdims.detach().cpu().numpy().round(3))
        if verbose>1:
            print('ref   ', np.array(scdim_ref)[:len(scdims)].round(3))

        opes={}
        for i,j,k in itt.product(range(3),repeat=3):
            opes[(i,j,k)]=get_ope_coeff(layers[-1],[ops[i],ops[j],ops[k]],
                                        [scdims[i],scdims[j],scdims[k]])

        if verbose>1:
            print('ope coeffs')
            for i,j,k in itt.combinations_with_replacement(range(3),3):
                for i1,j1,k1 in sorted(set(itt.permutations([i,j,k]))):
                    print('C_'+str(i1)+str(j1)+str(k1), '%.3f'%opes[(i1,j1,k1)].abs(), end='\t')
                print()
        else:
            print('C_112', '%.3f'%opes[(1,1,2)].abs())

# Our Code

In [4]:

J,g=1,1
hAB,hBA=get_Ising1D_Hamiltonians_2blocked(J,g)
print('J=%f, g=%f'%(J,g))
print('Exact energy=%f'%energy_ref)

filename='./data/Ising1DMERA.pth'
if not os.path.exists(os.path.dirname(filename)):
    os.makedirs(os.path.dirname(filename))
optionss=[
    MERAOptions(nLayers=2,max_dim=6,max_dim_mid=4,nSweeps=2000),
    MERAOptions(nLayers=3,max_dim=8,max_dim_mid=6,nSweeps=1800),
    MERAOptions(nLayers=4,max_dim=12,max_dim_mid=8,nSweeps=1400),
]
for ii,options in enumerate(optionss):
    print('sweep using the following options: ')
    print(asdict(options))
    if ii==0:
        layers=init_layers(hAB,hBA,options)
    else:
        pad_layers_(layers,options)
    for i in tqdm(range(1,1+options.nSweeps)):
        options.iSweep=i
        sweep_layers(layers,options)
        if i%(options.nSweeps//10)==0:
            print('Sweep',i)
            troubleshoot(layers,options,verbose=1)
            torch.save((layers,options), filename)
            print('saved to', filename)
    troubleshoot(layers,options,verbose=2)
    torch.save((layers,options), filename)
    print('saved to', filename)

J=1.000000, g=1.000000
Exact energy=-1.273240
sweep using the following options: 
{'reflection_symmetry': True, 'nLayers': 2, 'max_dim_mid': 4, 'max_dim': 6, 'nIterRhoTop': 4, 'nSweeps': 2000, 'iSweep': 0, 'freeze_u_before': 10}


  0%|          | 0/2000 [00:00<?, ?it/s]

Sweep 200
Energy -1.2725098 error 7.30e-04
scdims [0.    0.137 1.119 1.384 1.684 2.297 2.297 2.325 2.337 2.597]
C_112 0.675
saved to ./data/Ising1DMERA.pth
Sweep 400
Energy -1.2731918 error 4.78e-05
scdims [0.    0.133 1.11  1.203 1.236 2.019 2.034 2.072 2.179 2.323]
C_112 0.564
saved to ./data/Ising1DMERA.pth
Sweep 600
Energy -1.2732117 error 2.79e-05
scdims [0.    0.128 1.072 1.163 1.173 1.859 1.924 1.997 2.049 2.349]
C_112 0.527
saved to ./data/Ising1DMERA.pth
Sweep 800
Energy -1.2732145 error 2.50e-05
scdims [0.    0.127 1.063 1.154 1.16  1.847 1.906 1.983 2.028 2.376]
C_112 0.516
saved to ./data/Ising1DMERA.pth
Sweep 1000
Energy -1.2732159 error 2.36e-05
scdims [0.    0.127 1.062 1.153 1.158 1.857 1.905 1.979 2.027 2.393]
C_112 0.513
saved to ./data/Ising1DMERA.pth
Sweep 1200
Energy -1.2732169 error 2.27e-05
scdims [0.    0.127 1.062 1.153 1.159 1.872 1.907 1.976 2.03  2.404]
C_112 0.511
saved to ./data/Ising1DMERA.pth
Sweep 1400
Energy -1.2732175 error 2.20e-05
scdims [0.    0.12

  0%|          | 0/1800 [00:00<?, ?it/s]

Sweep 180
Energy -1.2732378 error 1.70e-06
scdims [0.    0.125 1.001 1.085 1.153 1.791 1.92  1.935 1.935 1.956]
C_112 0.500
saved to ./data/Ising1DMERA.pth
Sweep 360
Energy -1.2732381 error 1.47e-06
scdims [0.    0.125 0.999 1.083 1.151 1.792 1.918 1.933 1.933 1.953]
C_112 0.500
saved to ./data/Ising1DMERA.pth
Sweep 540
Energy -1.2732382 error 1.32e-06
scdims [0.    0.125 0.999 1.083 1.15  1.795 1.921 1.934 1.934 1.952]
C_112 0.500
saved to ./data/Ising1DMERA.pth
Sweep 720
Energy -1.2732383 error 1.21e-06
scdims [0.    0.125 0.999 1.084 1.15  1.798 1.923 1.936 1.936 1.952]
C_112 0.500
saved to ./data/Ising1DMERA.pth
Sweep 900
Energy -1.2732384 error 1.13e-06
scdims [0.    0.125 0.998 1.084 1.15  1.801 1.925 1.936 1.937 1.952]
C_112 0.500
saved to ./data/Ising1DMERA.pth
Sweep 1080
Energy -1.2732385 error 1.07e-06
scdims [0.    0.125 0.998 1.084 1.15  1.802 1.926 1.937 1.938 1.952]
C_112 0.500
saved to ./data/Ising1DMERA.pth
Sweep 1260
Energy -1.2732385 error 1.03e-06
scdims [0.    0.125

  0%|          | 0/1400 [00:00<?, ?it/s]

Sweep 140
Energy -1.2732395 error 8.29e-08
scdims [0.    0.125 0.999 1.125 1.129 1.483 1.609 1.751 1.902 1.902]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 280
Energy -1.2732395 error 6.64e-08
scdims [0.    0.125 0.999 1.124 1.128 1.803 1.803 1.849 1.849 1.975]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 420
Energy -1.2732395 error 5.79e-08
scdims [0.    0.125 0.999 1.121 1.127 1.887 1.887 1.891 1.891 1.958]
C_112 0.502
saved to ./data/Ising1DMERA.pth
Sweep 560
Energy -1.2732395 error 5.18e-08
scdims [0.    0.125 0.999 1.118 1.125 1.898 1.898 1.934 1.934 1.948]
C_112 0.501
saved to ./data/Ising1DMERA.pth
Sweep 700
Energy -1.2732395 error 4.65e-08
scdims [0.    0.125 0.998 1.117 1.122 1.899 1.899 1.941 1.941 1.941]
C_112 0.501
saved to ./data/Ising1DMERA.pth
Sweep 840
Energy -1.2732395 error 4.35e-08
scdims [0.    0.125 0.998 1.116 1.12  1.902 1.902 1.935 1.937 1.937]
C_112 0.501
saved to ./data/Ising1DMERA.pth
Sweep 980
Energy -1.2732395 error 4.14e-08
scdims [0.    0.125 0

In [ ]:
# Energy -1.2732395 error 3.73e-08
# scdims [0.    0.125 0.998 1.116 1.116 1.913 1.913 1.916 1.922 1.927]
# ref    [0.    0.125 1.    1.125 1.125 2.    2.    2.    2.    2.125]
# ope coeffs
# C_000 1.000	
# C_001 0.000	C_010 0.000	C_100 0.000	
# C_002 0.000	C_020 0.000	C_200 0.000	
# C_011 1.000	C_101 1.000	C_110 1.000	
# C_012 0.023	C_021 0.007	C_102 0.012	C_120 0.007	C_201 0.012	C_210 0.022	
# C_022 1.006	C_202 1.000	C_220 1.000	
# C_111 0.000	
# C_112 0.501	C_121 0.502	C_211 0.498	
# C_122 0.006	C_212 0.001	C_221 0.000	
# C_222 0.010	

# GE Energy -1.2732394 error 1.55e-07

# Compare with GE's code

In [23]:
def get_GE_Hamiltonians():
    sX = np.array([[0, 1], [1, 0]], dtype=float)
    sZ = np.array([[1, 0], [0, -1]], dtype=float)
    htemp = -np.kron(sX, sX) - 0.5 * (
        np.kron(sZ, np.eye(2)) + np.kron(np.eye(2), sZ))
    hbig = (0.5 * np.kron(np.eye(4), htemp) +
            np.kron(np.eye(2), np.kron(htemp, np.eye(2))) +
            0.5 * np.kron(htemp, np.eye(4))).reshape(2, 2, 2, 2, 2, 2, 2, 2)
    hamAB = (hbig.transpose(0, 1, 3, 2, 4, 5, 7, 6)).reshape(4, 4, 4, 4)
    hamBA = (hbig.transpose(1, 0, 2, 3, 5, 4, 6, 7)).reshape(4, 4, 4, 4)
    hamAB=torch.tensor(hamAB)/2
    hamBA=torch.tensor(hamBA)/2
    return hamAB,hamBA

def load_GE_data(path):
    ws, us, vs, rhoABs, rhoBAs=np.load(path,allow_pickle=True)
    print('dtype',ws[0].dtype)
    options=MERAOptions(
        nLayers=len(ws),
        max_dim=ws[-1].shape[2],
        max_dim_mid=ws[-1].shape[1])
    layers=[]
    hAB,hBA=get_GE_Hamiltonians()
    d=4
    h_bias=torch.maximum(
        torch.linalg.eigvalsh(hAB.reshape((d**2,d**2))).max(),
        torch.linalg.eigvalsh(hBA.reshape((d**2,d**2))).max()
    )
    hAB,hBA=hAB-h_bias*torch.eye(d**2).reshape((d,d,d,d)),hBA-h_bias*torch.eye(d**2).reshape((d,d,d,d))
    for i in range(len(ws)):
        w,v,u,rhoAB,rhoBA=ws[i],vs[i],us[i],rhoABs[i+1],rhoBAs[i+1]
        w,v,u,rhoAB,rhoBA=torch.tensor(w),torch.tensor(v),torch.tensor(u),torch.tensor(rhoAB),torch.tensor(rhoBA)
        w=contract('123->312',w)
        v=contract('123->321',v)
        u=contract('1234->3412',u)
        rhoAB=contract('1234->1234',rhoAB)
        rhoBA=contract('1234->1234',rhoBA)
        layers.append(MERALayer(w=w,u=u,v=v,rhoAB=rhoAB,rhoBA=rhoBA,hAB=hAB,hBA=hBA,h_bias=h_bias))
        hAB,hBA=propogate_up(layers[-1],options)
    return layers,options

layers_GE,options_GE=load_GE_data('./GEMeraDemo/IsingData.npy')
GE_filename='./data/GEMERA.pth'
torch.save((layers_GE,options_GE), GE_filename)
print('saved to', GE_filename)

dtype float64
saved to ./data/GEMERA.pth


In [24]:
layers,options=deepcopy(layers_GE),deepcopy(options_GE)


rhoAB0,rhoBA0=layers[-1].rhoAB.clone(),layers[-1].rhoBA.clone()
for i in range(10):
    layers[-1].rhoAB,layers[-1].rhoBA=propogate_down(layers[-1],options)
    rhoAB,rhoBA=layers[-1].rhoAB,layers[-1].rhoBA
    print('%.2e'%torch.norm(rhoAB-rhoAB0).item(),'%.2e'%torch.norm(rhoBA-rhoBA0).item())



1.09e-06 9.47e-07
9.92e-07 1.08e-06
1.12e-06 1.16e-06
1.03e-06 1.09e-06
1.11e-06 1.15e-06
1.04e-06 1.09e-06
1.11e-06 1.15e-06
1.04e-06 1.09e-06
1.10e-06 1.14e-06
1.04e-06 1.09e-06


In [26]:

layers=deepcopy(layers_GE)
options=deepcopy(options_GE)

troubleshoot(layers,options,verbose=2)
options.nSweeps=100
for i in tqdm(range(1,1+options.nSweeps)):
    sweep_layers(layers,options)
    if i%20==0 or i<=4:
        print('Sweep',i)
        troubleshoot(layers,options,verbose=1)

Energy -1.2732394 error 1.55e-07
scdims [0.    0.125 1.005 1.12  1.13  1.763 1.856 1.949 1.965 2.082]
ref    [0.    0.125 1.    1.125 1.125 2.    2.    2.    2.    2.125]
ope coeffs
C_000 1.000	
C_001 0.000	C_010 0.000	C_100 0.000	
C_002 0.000	C_020 0.000	C_200 0.000	
C_011 0.999	C_101 1.000	C_110 1.000	
C_012 0.092	C_021 0.026	C_102 0.050	C_120 0.026	C_201 0.048	C_210 0.092	
C_022 0.989	C_202 1.000	C_220 0.997	
C_111 0.000	
C_112 0.500	C_121 0.501	C_211 0.504	
C_122 0.030	C_212 0.013	C_221 0.019	
C_222 0.003	


  0%|          | 0/100 [00:00<?, ?it/s]

Sweep 1
Energy -1.2732394 error 1.55e-07
scdims [0.    0.125 1.005 1.12  1.13  1.763 1.856 1.949 1.965 2.082]
C_112 0.500
Sweep 2
Energy -1.2732394 error 1.55e-07
scdims [0.    0.125 1.005 1.12  1.13  1.763 1.856 1.949 1.965 2.082]
C_112 0.500
Sweep 3
Energy -1.2732394 error 1.55e-07
scdims [0.    0.125 1.005 1.12  1.13  1.763 1.856 1.949 1.965 2.082]
C_112 0.500
Sweep 4
Energy -1.2732394 error 1.55e-07
scdims [0.    0.125 1.005 1.12  1.13  1.763 1.856 1.949 1.965 2.082]
C_112 0.500
Sweep 20
Energy -1.2732394 error 1.54e-07
scdims [0.    0.125 1.005 1.12  1.13  1.765 1.858 1.95  1.966 2.082]
C_112 0.500
Sweep 40
Energy -1.2732394 error 1.54e-07
scdims [0.    0.125 1.005 1.12  1.13  1.768 1.86  1.95  1.967 2.082]
C_112 0.500
Sweep 60
Energy -1.2732394 error 1.53e-07
scdims [0.    0.125 1.005 1.12  1.13  1.771 1.861 1.951 1.968 2.082]
C_112 0.500
Sweep 80
Energy -1.2732394 error 1.53e-07
scdims [0.    0.125 1.005 1.12  1.13  1.774 1.864 1.952 1.969 2.082]
C_112 0.500


KeyboardInterrupt: 

2